<a href="https://colab.research.google.com/github/nallagondu/DATASCIENCE-practice/blob/featurebranch/GICC_INC_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from wordcloud import WordCloud, STOPWORDS
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from google.colab import drive
from matplotlib import pyplot as plt


In [ ]:

drive.mount('/content/drive')

In [ ]:
# Load the data from Google Drive
data_path = '/content/drive/MyDrive/Colab Notebooks/DATA for Colab/incident52000.csv'
df = pd.read_csv(data_path)
df.head()

In [ ]:
df.tail()


In [ ]:
df.shape


In [ ]:
df.describe()

In [ ]:
missing_values = df.isnull().sum()
missing_values

In [ ]:
df.drop_duplicates()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns=['Activity due','Work start','Timeline','Additional assignee list', 'Approval set', 'Due Date', 'SLA due'], inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
mode_assigned_to = df['Assigned to'].mode()[0]  # Calculate mode (most frequent value)
df['Assigned to'].fillna(mode_assigned_to, inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
# Function to extract server name
def extract_server_name(description):
    parts = description.split('_-_')
    server_name = parts[-1].strip()
    server_name = server_name.replace('[Server]', '').strip()
    return server_name

# Apply the function to extract server names only for rows where 'Causal Configuration Item' is empty
mask = df['Causal Configuration Item'].isnull()
df.loc[mask, 'Causal Configuration Item'] = df.loc[mask, 'Short Description'].apply(extract_server_name)

# Print the DataFrame with the extracted server names filled into the 'Causal Configuration Item' column
print(df)

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
# Fill empty values in 'Resolved' with corresponding values from 'Created'
df['Resolved'].fillna(df['Created'], inplace=True)
df

In [ ]:
df.isnull().sum()

In [ ]:
df.head()


In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Extract server name from 'Short Description' column
df['Server'] = df['Causal Configuration Item'].str.split('_-_').str[-1].str.strip()

# Encode categorical variables
label_encoder = LabelEncoder()
df['Server'] = label_encoder.fit_transform(df['Server'])
# Save the DataFrame to cleaned.csv
df.to_csv('cleanedData.csv', index=False)
df

In [ ]:
df['Created by'] = label_encoder.fit_transform(df['Created by'])
df['Severity'] = label_encoder.fit_transform(df['Severity'])

df.head()

In [ ]:
# Fill missing values in categorical columns with the most frequent value
categorical_cols = ['Caller', 'Causal Configuration Item', 'Description', 'Priority', 'Assigned to', 'State', 'Assignment group', 'Server']
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

In [ ]:
# Fill missing values in numerical columns with median or mean
numerical_cols = ['Business duration', 'Child Incidents']
for col in numerical_cols:
    df[col].fillna(df[col].median(), inplace=True)
df.head()

In [ ]:
# Convert categorical variables into numerical representations using label encoding
#label_encoder = LabelEncoder()
#for col in categorical_cols:
  #  df[col] = label_encoder.fit_transform(df[col])

# Encode categorical columns: Caller, Server
label_encoder = LabelEncoder()
df['Caller'] = label_encoder.fit_transform(df['Caller'])
df

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def preprocess_text(text):
    # Check if the input is a list
    if isinstance(text, list):
        # Join list elements into a single string
        text = ' '.join(text)
    # Convert to lowercase
    text = text.lower()
    return text

In [ ]:
# Apply text preprocessing
df['Short Description'] = df['Short Description'].apply(preprocess_text)
df['Description'] = df['Description'].apply(preprocess_text)

In [ ]:
# Process Short Description column by splitting into chunks
# Define a function to split text into chunks
#def split_into_chunks(text, chunk_size=50):
 #   chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
 #   return chunks

In [ ]:
# Apply the function to Short Description column
#df['Short Description'] = df['Short Description'].apply(split_into_chunks)

# Check the updated DataFrame
#print(df.head())

In [ ]:
# Function to perform retrieval and count occurrences for a specific server or based on description
def retrieve_and_count_occurrences(query, index, vectorizer, dataset):
    query_vector = vectorizer.transform([preprocess_text(query)]).toarray()

    # Calculate cosine similarity between query vector and vectors in the index
    similarity_scores = {}
    for i, vector in index.items():
        similarity = cosine_similarity(query_vector, [vector])[0][0]
        similarity_scores[i] = similarity

    # Retrieve top similar documents for the specific server
    top_documents = sorted(similarity_scores, key=similarity_scores.get, reverse=True)[:len(similarity_scores)]
    top_documents_info = [(dataset.iloc[i]['Causal Configuration Item'], dataset.iloc[i]['Description']) for i in top_documents if query.lower() in dataset.iloc[i]['Causal Configuration Item'].lower()]

    # If no logs found for the given server, analyze based on Description and Short Description
    if not top_documents_info:
        # Analyze based on Description and Short Description
        similar_issues = {}
        for desc in dataset['Description']:
            if query.lower() in desc.lower():
                if desc in similar_issues:
                    similar_issues[desc] += 1
                else:
                    similar_issues[desc] = 1
        for short_desc in dataset['Short Description']:
            if query.lower() in short_desc.lower():
                if short_desc in similar_issues:
                    similar_issues[short_desc] += 1
                else:
                    similar_issues[short_desc] = 1

        if not similar_issues:
            return "No logs or similar issues found for the given server or description.", None

        most_repeated_issue = max(similar_issues, key=similar_issues.get)
        return "No logs found for the given server. Analyzing based on Description and Short Description.", most_repeated_issue

    # Analyze repeated issues for the specific server
    issues_count = {}
    for _, desc in top_documents_info:
        if query.lower() in desc.lower():
            if desc in issues_count:
                issues_count[desc] += 1
            else:
                issues_count[desc] = 1

    # Find the most repeated issue for the specific server
    most_repeated_issue = max(issues_count, key=issues_count.get)

    return len(top_documents_info), most_repeated_issue

# Example of retrieval and occurrences count for a specific server or based on description
query = "MESPWERIAT03"  # Replace with your query

result = retrieve_and_count_occurrences(query, short_description_index, tfidf_vectorizer, df)
if isinstance(result, tuple):
    occurrences_count, most_repeated_issue = result
    print(f"Occurrences count for server '{query}': {occurrences_count}")
    print(f"Most repeated issue for server '{query}': {most_repeated_issue}")
else:
    print(result)  # No logs or similar issues found message


In [ ]:
# Function to perform retrieval and count occurrences for a specific server or based on description
def retrieve_and_count_occurrences(query, index, vectorizer, dataset):
    query_vector = vectorizer.transform([preprocess_text(query)]).toarray()

    # Calculate cosine similarity between query vector and vectors in the index
    similarity_scores = {}
    for i, vector in index.items():
        similarity = cosine_similarity(query_vector, [vector])[0][0]
        similarity_scores[i] = similarity

    # Retrieve top similar issue for the specific server
    top_documents = sorted(similarity_scores, key=similarity_scores.get, reverse=True)[:len(similarity_scores)]
    top_documents_info = [(dataset.iloc[i]['Causal Configuration Item'], dataset.iloc[i]['Description']) for i in top_documents if query.lower() in dataset.iloc[i]['Causal Configuration Item'].lower()]

    # If no logs found for the given server, analyze based on Description and Short Description
    if not top_documents_info:
        # Analyze based on Description and Short Description
        similar_issues = {}
        for desc in dataset['Description']:
            if query.lower() in desc.lower():
                if desc in similar_issues:
                    similar_issues[desc] += 1
                else:
                    similar_issues[desc] = 1
        for short_desc in dataset['Short Description']:
            if query.lower() in short_desc.lower():
                if short_desc in similar_issues:
                    similar_issues[short_desc] += 1
                else:
                    similar_issues[short_desc] = 1

        if not similar_issues:
            return "No logs or similar issues found for the given server or description.", None

        most_repeated_issue = max(similar_issues, key=similar_issues.get)
        return "No logs found for the given server. Analyzing based on Description and Short Description.", similar_issues

    # Analyze repeated issues for the specific server
    issues_count = {}
    for _, desc in top_documents_info:
        if query.lower() in desc.lower():
            if desc in issues_count:
                issues_count[desc] += 1
            else:
                issues_count[desc] = 1

    # Find the most repeated issue for the specific server
    most_repeated_issue = max(issues_count, key=issues_count.get)

    return len(top_documents_info), issues_count, most_repeated_issue

# Example of retrieval and occurrences count for a specific server or based on description
query = "MESPWERIAT03"  # Replace with your query

result = retrieve_and_count_occurrences(query, short_description_index, tfidf_vectorizer, df)
if isinstance(result, tuple):
    occurrences_count, issues_count, most_repeated_issue = result
    print(f"Occurrences count for server '{query}': {occurrences_count}")
    print(f"Most repeated issue for server '{query}': {most_repeated_issue}")
    print("Issues Count:")
    for issue, frequency in issues_count.items():
        print(f"Issue: {issue}, Frequency: {frequency}")
else:
    print(result)  # No logs or similar issues found message


# Example of retrieval and occurrences count for a specific server or based on description
# Please enter the query details in query = "<xxxxxx>"  # Replace with your query


In [ ]:
from collections import defaultdict

# Function to perform retrieval and count occurrences for a specific server or based on description
def retrieve_and_count_occurrences(query, index, vectorizer, dataset):
    query_vector = vectorizer.transform([preprocess_text(query)]).toarray()

    # Calculate cosine similarity between query vector and vectors in the index
    similarity_scores = {}
    for i, vector in index.items():
        similarity = cosine_similarity(query_vector, [vector])[0][0]
        similarity_scores[i] = similarity

    # Retrieve top similar issue for the specific server
    top_documents = sorted(similarity_scores, key=similarity_scores.get, reverse=True)[:len(similarity_scores)]
    top_documents_info = [(dataset.iloc[i]['Causal Configuration Item'], dataset.iloc[i]['Description'], dataset.iloc[i]['Created']) for i in top_documents if query.lower() in dataset.iloc[i]['Causal Configuration Item'].lower()]

    # If no logs found for the given server, analyze based on Description and Short Description
    if not top_documents_info:
        # Analyze based on Description and Short Description
        similar_issues = defaultdict(list)
        for i, desc in enumerate(dataset['Description']):
            if query.lower() in desc.lower():
                similar_issues[desc].append((dataset.iloc[i]['Causal Configuration Item'], dataset.iloc[i]['Created']))
        for i, short_desc in enumerate(dataset['Short Description']):
            if query.lower() in short_desc.lower():
                similar_issues[short_desc].append((dataset.iloc[i]['Causal Configuration Item'], dataset.iloc[i]['Created']))

        if not similar_issues:
            return "No logs or similar issues found for the given server or description.", None

        most_repeated_issue = max(similar_issues, key=lambda x: len(similar_issues[x]))
        return "No logs found for the given server. Analyzing based on Description and Short Description.", similar_issues[most_repeated_issue], most_repeated_issue

    # Analyze repeated issues for the specific server
    issues_count = {}
    for _, desc, created in top_documents_info:
        if query.lower() in desc.lower():
            if desc in issues_count:
                issues_count[desc].append(created)
            else:
                issues_count[desc] = [created]

    # Find the most repeated issue for the specific server
    most_repeated_issue = max(issues_count, key=lambda x: len(issues_count[x]))

    return len(top_documents_info), issues_count[most_repeated_issue], most_repeated_issue

# Example of retrieval and occurrences count for a specific server or based on description
query = "RDU1PWCULBOT21"  # Replace with your query

result = retrieve_and_count_occurrences(query, short_description_index, tfidf_vectorizer, df)
if isinstance(result, tuple):
    occurrences_count, most_repeated_issue, most_repeated_issue_desc = result
    print(f"Occurrences count for '{query}': {occurrences_count}")
    print(f"Most repeated issue for '{query}': {most_repeated_issue}")
    print("Issues Count:")
    for issue, frequency in issues_count.items():
        print(f"Issue: {issue}, Frequency: {frequency}")
else:
    print(result)  # No logs or similar issues found message


**method 4**
 ##  to find the data accurancy based on any word or server name and it count the number of accurancy

In [ ]:
 #is to find the data accurancy based on any word or server name and it count the number of accurancy
def find_word_in_dataframe(word, dataframe):
    occurrences_count = 0
    matching_rows = []

    # Iterate through each row in the DataFrame
    for index, row in dataframe.iterrows():
        # Check if the word is present in any of the relevant columns
        if (word.lower() in str(row['Causal Configuration Item']).lower() or
                word.lower() in str(row['Description']).lower() or
                word.lower() in str(row['Short Description']).lower() or
                word.lower() in str(row['Created']).lower()):  # Assuming 'Created' is the column name for date
            occurrences_count += 1
            matching_rows.append((row['Causal Configuration Item'], row['Description'], row['Short Description'], row['Created']))

    return occurrences_count, matching_rows

# Example usage
word = "access "  # Replace with the word you want to search for
occurrences_count, matching_rows = find_word_in_dataframe(word, df)

print(f"Occurrences count for '{word}': {occurrences_count}")
if matching_rows:
    print("Matching Rows:")
    for row in matching_rows:
        print(row)
else:
    print("No matches found.")
